In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from pymongo import MongoClient

In [4]:
client = MongoClient('mongodb://localhost:27017/')
db = client['movie-web']
collection = db['userratings']
cursor = collection.find()
list_cur = []
for i in cursor:
    list_cur.append(i)
ratings = pd.DataFrame(list_cur)
ratings = ratings.drop(['_id', '__v', 'createdAt', 'updatedAt'], axis=1)
ratings

,movie,user,rating
0,633dc3b4f611a788116b43d8,633e8ba2bc3a40391db45a5c,5
1,633dc3b4f611a788116b43d8,633e900bbc3a40391db45a7f,5
2,633dc52cf611a788116b43dc,633b30e4edfee7840d1006c6,4
3,633dc52cf611a788116b43dc,633e8ba2bc3a40391db45a5c,5
4,633dd3e4f611a788116b43e7,633e8ba2bc3a40391db45a5c,2
5,633dd3e4f611a788116b43e7,633b30e4edfee7840d1006c6,2
6,633dc52cf611a788116b43dc,633e900bbc3a40391db45a7f,3
7,6354ebb6c834f95698ef02ca,633e8ba2bc3a40391db45a5c,2


In [45]:
# ratings = pd.read_csv('ratings.csv')
# movies = pd.read_csv('movies.csv')

In [46]:
# ratings.head()

In [47]:
# movies.head()

In [48]:
# print(movies.shape)

In [5]:
userRatings = ratings.pivot_table(index=['movie'],columns=['user'],values='rating')
# userRatings = userRatings.dropna(thresh=10, axis=0)
userRatings.head()

user,633b30e4edfee7840d1006c6,633e8ba2bc3a40391db45a5c,633e900bbc3a40391db45a7f
movie,,,
633dc3b4f611a788116b43d8,NaN,5.0,5.0
633dc52cf611a788116b43dc,4.0,5.0,3.0
633dd3e4f611a788116b43e7,2.0,2.0,NaN
6354ebb6c834f95698ef02ca,NaN,2.0,NaN


In [6]:
def standardize(col):
    new_col = (col - col.mean())
    return new_col

df_std = userRatings.apply(standardize, axis = 0)
df_std.fillna(0, inplace=True)
df_std

user,633b30e4edfee7840d1006c6,633e8ba2bc3a40391db45a5c,633e900bbc3a40391db45a7f
movie,,,
633dc3b4f611a788116b43d8,0.0,1.5,1.0
633dc52cf611a788116b43dc,1.0,1.5,-1.0
633dd3e4f611a788116b43e7,-1.0,-1.5,0.0
6354ebb6c834f95698ef02ca,0.0,-1.5,0.0


In [66]:
sparse_df = sparse.csr_matrix(df_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df.T),index=userRatings.columns,columns=userRatings.columns)
corrMatrix

user,633b30e4edfee7840d1006c6,633e8ba2bc3a40391db45a5c,633e900bbc3a40391db45a7f
user,,,
633b30e4edfee7840d1006c6,1.000000,0.594089,-0.5
633e8ba2bc3a40391db45a5c,0.594089,1.000000,0.0
633e900bbc3a40391db45a7f,-0.500000,0.000000,1.0


In [67]:
def pred(u, i):
    ids = np.where(ratings.iloc[:, 0] == i)[0]
    users_rated_i = (ratings.iloc[ids, 1])
    sim = corrMatrix.loc[u, users_rated_i]
    a = sim.sort_values().iloc[-2:]
    idx = a.index
    r = sparse_df[df_std.index.get_loc(i), df_std.columns.get_indexer(idx)]
    return (r * a)[0] / (np.abs(a).sum() + 1e-8)
def recommend_top(u, top_x):
    ids = np.where(ratings.iloc[:, 1] == u)[0]
    items_rated_by_u = ratings.iloc[ids, 0].tolist()
    items = df_std.index.tolist()
    item = {'userId': None, 'movieId': None, 'similar': None}
    list_items = []
    def take_similar(elem):
            return elem['similar']
    for i in items:
        if i not in items_rated_by_u:
            rating = pred(u, i)
            item['userId'] = u
            item['movieId'] = i
            item['similar'] = rating
            list_items.append(item.copy())  
    sorted_items = sorted(list_items, key=take_similar, reverse=True)
    sorted_items = sorted_items[:top_x]
    return sorted_items
df = pd.DataFrame(columns=['userId', 'movieId', 'similar'])
users = df_std.columns.tolist()
for u in users:
    df = pd.concat([df, pd.DataFrame(recommend_top(u, 10))])
df.to_csv(r'C:\Users\english\Documents\FinalProject\api\recommend.csv', index=False)

,userId,movieId,similar
0,633b30e4edfee7840d1006c6,633dc3b4f611a788116b43d8,0.493246
1,633b30e4edfee7840d1006c6,6354ebb6c834f95698ef02ca,-2.250000
0,633e900bbc3a40391db45a7f,633dd3e4f611a788116b43e7,1.000000
1,633e900bbc3a40391db45a7f,6354ebb6c834f95698ef02ca,0.000000


In [6]:
recommend = pd.read_csv('recommend.csv')
ids = np.where(recommend.iloc[:, 0] == '633b30e4edfee7840d1006c6')[0]
movies = recommend.iloc[ids, 1].tolist()
str(movies)

,userId,movieId,similar
0,633b30e4edfee7840d1006c6,6354ec56c834f95698ef02cd,1.200000
1,633b30e4edfee7840d1006c6,633dc3b4f611a788116b43d8,0.239718
2,633b30e4edfee7840d1006c6,6354ebb6c834f95698ef02ca,-1.800000
3,633e900bbc3a40391db45a7f,633dd3e4f611a788116b43e7,1.000000
4,633e900bbc3a40391db45a7f,6354ebb6c834f95698ef02ca,0.000000
5,633e900bbc3a40391db45a7f,6354ec56c834f95698ef02cd,0.000000


In [7]:
sparse_df = sparse.csr_matrix(df_std.values)
corrMatrix = pd.DataFrame(cosine_similarity(sparse_df),index=userRatings.index,columns=userRatings.index)
corrMatrix

movie,633dc3b4f611a788116b43d8,633dc52cf611a788116b43dc,633dd3e4f611a788116b43e7,6354ebb6c834f95698ef02ca
movie,,,,
633dc3b4f611a788116b43d8,1.000000,0.336336,-0.692308,-0.832050
633dc52cf611a788116b43dc,0.336336,1.000000,-0.874475,-0.727607
633dd3e4f611a788116b43e7,-0.692308,-0.874475,1.000000,0.832050
6354ebb6c834f95698ef02ca,-0.832050,-0.727607,0.832050,1.000000


In [85]:
corrMatrix.to_csv(r'C:\Users\english\Documents\FinalProject\api\movieToMovie.csv', index = True)

In [8]:
similarity_scores = pd.read_csv('movieToMovie.csv', index_col=0)
similarity_scores

,633dc3b4f611a788116b43d8,633dc52cf611a788116b43dc,633dd3e4f611a788116b43e7,6354ebb6c834f95698ef02ca
movie,,,,
633dc3b4f611a788116b43d8,1.000000,0.336336,-0.692308,-0.832050
633dc52cf611a788116b43dc,0.336336,1.000000,-0.874475,-0.727607
633dd3e4f611a788116b43e7,-0.692308,-0.874475,1.000000,0.832050
6354ebb6c834f95698ef02ca,-0.832050,-0.727607,0.832050,1.000000


In [16]:
def recommend(book):
    items = similarity_scores.index.tolist()
    idx = []
    if book in items:
        scrs = similarity_scores.loc[book]
        a = scrs.sort_values(ascending = False)[1:10]
        idx = a.index.tolist()
    return idx
recommend('6354e362c834f95698ef0221')

[]